## Состав портфолио

In [2]:
%load_ext autoreload
%autoreload 2


from functools import reduce

TOTAL = 1

# by type
shares_by_type = {
    "stocks": 0.75,
    "obligations": 0.25
}

# by region
stocks_by_region = {
    "america": 0.7,
    "foreign": 0.3
}

# by cap
stocks_by_cap = {
    "large": 0.7,
    "small": 0.3
}

def assert_shares(obj):
    assert reduce(lambda a,b: a + b, obj.values()) == TOTAL
    
assert_shares(shares_by_type)
assert_shares(stocks_by_region)
assert_shares(stocks_by_cap)

## Плоское портфолио акций

In [3]:
flatten_stock_portfolio = []

stock_share = shares_by_type["stocks"]

for (region_key, region_value) in stocks_by_region.items():
    for (cap_key, cap_value) in stocks_by_cap.items():
        flatten_stock_portfolio.append({
            "region": region_key,
            "share": stock_share * region_value * cap_value,
            "cap": cap_key
        })
        
def sort_by_share(portfolio):
    return sorted(portfolio, key=lambda x: x["share"], reverse=True)
        
flatten_stock_portfolio = sort_by_share(flatten_stock_portfolio)

In [4]:
import pandas as pd

def share_to_percent(share):
    return f'{round(share * 100, 2)}%'

def prepare_stock_table_rows(flatten_portfolio):
    stocks_table = []
    
    full_share = 0
    for row in flatten_portfolio:
        full_share += row["share"]
    
    for stock in flatten_portfolio:
        share = stock["share"]
        stocks_table.append([
            share_to_percent(share / full_share),
            share_to_percent(share)
        ])

    return stocks_table
    
def get_stock_name(share):
    return f'{share["region"].capitalize()} {share["cap"].capitalize()}-cap stocks'

stocks_table = prepare_stock_table_rows(flatten_stock_portfolio)
    
df = pd.DataFrame(stocks_table, columns=['Share', 'Share of all'], index=[get_stock_name(x) for x in flatten_stock_portfolio])
df

,Share,Share of all
America Large-cap stocks,49.0%,36.75%
America Small-cap stocks,21.0%,15.75%
Foreign Large-cap stocks,21.0%,15.75%
Foreign Small-cap stocks,9.0%,6.75%


## Historical data

Data have been taken from http://www.lazyportfolioetf.com .

### Stocks
- [Russell 2000](http://www.lazyportfolioetf.com/etf/ishares-russell-2000-etf-iwm/) - American Small-Cap market ETF
- [VSS](http://www.lazyportfolioetf.com/etf/vanguard-ftse-all-world-ex-us-small-cap-etf-vss) - FTSE All-World ex-US Small-Cap ETF
- [EFA](http://www.lazyportfolioetf.com/etf/ishares-msci-eafe-efa/) - ex-US Large-Cap market ETF tracking EAFE
- [VTI](http://www.lazyportfolioetf.com/etf/vanguard-total-stock-market-vti/) - CRSP US Total Market Index
- [VNQ](http://www.lazyportfolioetf.com/etf/vanguard-real-estate-vnq/) - US REIT ETF
- GLD - Gold
- DBC - Commodities

### Bonds
- [BND](http://www.lazyportfolioetf.com/etf/vanguard-total-bond-market-bnd/) - Vanguard Total Bond Market
- [SHY](http://www.lazyportfolioetf.com/etf/ishares-1-3-year-treasury-bond-shy/) - iShares 1-3 Year Treasury Bond

# Analytics

In [18]:
from collections import OrderedDict
from src.statistics import get_max_first_year
from src.laxy_portfolio_etf import get_returns_data

stocks_data = OrderedDict({
    "VTI": get_returns_data("data/monthly-returns/vti-returns-monthly.txt"),
    "EAFE": get_returns_data("data/monthly-returns/eafe-returns-monthly.txt"),
    "Russell 2000": get_returns_data("data/monthly-returns/russell_2000-returns-monthly.txt"),
    "EAFE Small-Cap": get_returns_data("data/monthly-returns/eafe-small-cap-return-monthly.txt"),
    "VNQ (REIT)": get_returns_data("data/monthly-returns/vnq-returns.txt"),
    "GLD": get_returns_data("data/monthly-returns/gld-returns.txt"),
    "DBC": get_returns_data("data/monthly-returns/dbc-returns.txt"),
    "BND": get_returns_data("data/monthly-returns/bnd-returns.txt"),
    "SHY": get_returns_data("data/monthly-returns/shy-returns.txt"),
})

max_first_year = get_max_first_year(stocks_data.values())
print("max_first_year", max_first_year)

max_first_year 1992


## Monthly returns plot

In [19]:
import numpy as np
import plotly.express as px
from src.statistics import get_max_returns_from_same_year, get_max_first_year, calc_returns_amount_list
from src.plot_utils import get_monthly_data_range

max_first_year = get_max_first_year(stocks_data.values())
init_amount = 1000
data = [calc_returns_amount_list(init_amount, x) for x in get_max_returns_from_same_year(stocks_data.values())]

df = pd.DataFrame(
    np.array(data).transpose(),
    columns=list(stocks_data.keys()), 
    index=get_monthly_data_range(max_first_year, 2021)
)

fig = px.line(df, x=df.index, y=df.columns)
fig.show()

## Correlations

In [29]:
import numpy as np
from src.statistics import get_correlation_table_view, get_stocks_std_view
    
stock_names = list(stocks_data.keys())
corr_table = get_correlation_table_view(stocks_data)
std_row = get_stocks_std_view(stocks_data)

index = stock_names
columns = ["std", "From", "|", *stock_names]

splitter_column = ["|"] * len(index)
from_column = [x.first_year for x in stocks_data.values()]

df = pd.DataFrame(
    np.array([std_row, from_column, splitter_column, *corr_table]).transpose(), 
    columns=columns,
    index=index
)
df

,std,From,|,VTI,EAFE,Russell 2000,EAFE Small-Cap,VNQ (REIT),GLD,DBC,BND,SHY
VTI,15.55%,1972,|,-,70.62%,89.34%,73.6%,61.69%,1.88%,17.08%,16.64%,4.38%
EAFE,17.4%,1986,|,70.62%,-,62.51%,85.57%,50.36%,12.83%,32.65%,9.71%,-8.3%
Russell 2000,19.74%,1985,|,89.34%,62.51%,-,71.96%,62.29%,-3.13%,24.19%,0.76%,-19.11%
EAFE Small-Cap,16.47%,1992,|,73.6%,85.57%,71.96%,-,57.33%,18.85%,47.89%,5.91%,-21.97%
VNQ (REIT),17.09%,1972,|,61.69%,50.36%,62.29%,57.33%,-,5.77%,14.96%,18.19%,3.73%
GLD,19.9%,1972,|,1.88%,12.83%,-3.13%,18.85%,5.77%,-,27.34%,7.79%,7.08%
DBC,18.7%,1971,|,17.08%,32.65%,24.19%,47.89%,14.96%,27.34%,-,-3.46%,-1.32%
BND,5.3%,1972,|,16.64%,9.71%,0.76%,5.91%,18.19%,7.79%,-3.46%,-,87.64%
SHY,3.03%,1977,|,4.38%,-8.3%,-19.11%,-21.97%,3.73%,7.08%,-1.32%,87.64%,-
